### Расстояние Левенштейна

<img src ='https://wikimedia.org/api/rest_v1/media/math/render/svg/bb9db0430ed76b6826156bbafc02d1d0183eec8c'/>

m(a, b) = 0 если a = b и = 1 иначе

#### Алгоритм

для всех i от 0 до M

   для всех j от 0 до N
   
     вычислить D(i, j)
     
 вернуть D(M, N)

In [1]:
import numpy as np

In [58]:
def levenshtein_distance(str1, str2):
    m, n = len(str1), len(str2)
    d1 = np.arange(m + 1)[np.newaxis]
    d2 = np.arange(1, n + 1)[:, np.newaxis]
    d3 = np.zeros((n, m))
    d = np.vstack((d1, np.hstack((d2, d3))))
    for i in range(m):
        for j in range(n):
            d[j + 1, i + 1] = np.min([d[j, i + 1] + 1, d[j + 1, i] + 1, d[j, i] + int(str1[i] != str2[j])])
    return d # [n, m]

In [59]:
levenshtein_distance('Saturday', 'Sunday')

array([[0., 1., 2., 3., 4., 5., 6., 7., 8.],
       [1., 0., 1., 2., 3., 4., 5., 6., 7.],
       [2., 1., 1., 2., 2., 3., 4., 5., 6.],
       [3., 2., 2., 2., 3., 3., 4., 5., 6.],
       [4., 3., 3., 3., 3., 4., 3., 4., 5.],
       [5., 4., 3., 4., 4., 4., 4., 3., 4.],
       [6., 5., 4., 4., 5., 5., 5., 4., 3.]])

In [38]:
str1 = 'Saturday'
str1[0]

'S'

In [39]:
m, n = 12, 4
d1 = np.arange(m + 1)[np.newaxis]
d2 = np.arange(1, n + 1)[:, np.newaxis]
d3 = np.zeros((n, m))

In [40]:
d1, d2, d3

(array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]]),
 array([[1],
        [2],
        [3],
        [4]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))

In [41]:
d = np.vstack((d1, np.hstack((d2, d3))))

In [42]:
d

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [47]:
d[0, 7]

7.0

In [60]:
import pandas as pd

In [75]:
df = pd.read_csv('queries_all.txt', sep='\t',  header=None)

In [78]:
df.head()

,0
0,видео секса смотреть бесплатно
1,шоплифтеры казань
2,платья 2016 фото новинки
3,вк
4,александр липовой


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991771 entries, 0 to 1991770
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       object
dtypes: object(1)
memory usage: 15.2+ MB


In [98]:
len(set('ABCDEFGHIJKLMNOPQRSTUVWXYZ'
        'abcdefghijklmnopqrstuvwxyz'
        'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ'
        'абвгдеёжзийклмнопрстуфхцчшщьыъэюя'))

118

In [100]:
def split(line):
    split_line = []
    letters = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ'
                  'abcdefghijklmnopqrstuvwxyz'
                  'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ'
                  'абвгдеёжзийклмнопрстуфхцчшщьыъэюя')

    if not line:
        return split_line

    line = line.lower()

    # word consists of letters
    word = ''
    for c in line:
        if c in letters:
            word += c
        else:
            if word:
                split_line.append(word)
                word = ''
    return split_line

In [102]:
split('enrique iglesias duele el corazon скачать бесплатно')

['enrique', 'iglesias', 'duele', 'el', 'corazon', 'скачать']

In [106]:
for line in df.values:
    if '\t' in line[0]:
        print('yes')
        break

yes


In [107]:
line = 'enrique iglesias duele el corazon скачать бесплатно'

In [113]:
line.find('3')

-1

In [114]:
from collections import defaultdict

In [115]:
d = defaultdict(float)

In [116]:
d['d'] += 3

In [117]:
d['s'] += 4

In [118]:
len(d)

2

In [119]:
d

defaultdict(float, {'d': 3.0, 's': 4.0})

In [123]:
for i in d:
    d[i] += 1

In [124]:
d

defaultdict(float, {'d': 4.0, 's': 5.0})

In [125]:
d['d'] /= 3 * 2
d

defaultdict(float, {'d': 0.6666666666666666, 's': 5.0})